In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

from sklearn import cluster
from scipy.cluster import hierarchy
from sklearn.cluster import DBSCAN
from currency_converter import CurrencyConverter

ModuleNotFoundError: No module named 'currency_converter'

### Preprocessing

In [7]:
# First Read it in
movies_df = pd.read_csv('IMDb movies.csv')

# Rename Columns and convert data types where appropriate
movies_df = movies_df.astype(
    {
     'duration' : 'float',
     'avg_vote' : 'float',
     'votes' : 'float',
#      'budget' : 'float', # Need this to replace '$ \d+' with float(\d+)
#      'usa_gross_income' : 'float', # Need this to replace '$ \d+' with float(\d+)
#      'worlwide_gross_income' : 'float', # Need this to replace '$ \d+' with float(\d+)
     'metascore' : 'float',
     'reviews_from_users' : 'float',
     'reviews_from_critics' : 'float'
    }).rename(columns=
    {'imdb_title_id' : 'title_id',
     'reviews_from_users' : 'user_reviews',
     'reviews_from_critics' : 'critic_reviews',
     'worlwide_gross_income' : 'ww_gross_income'
    })

movies_df = movies_df.set_index(movies_df['title_id']).rename(columns={'title_id' : 'REMOVE'}).drop(columns=['REMOVE'])

# Found a bad row that didn't contain an int for a year (or a year formatted as such)
movies_df = movies_df.drop(index=['tt8206668'], axis=1)
movies_df = movies_df.astype({'year' : 'int'})

# We also don't want any older films to keep the data relatively current
movies_df = movies_df[movies_df['year'] > 1969]

# Only consider films with over 10000 votes
movies_df = movies_df[movies_df['votes'] >= 1000]

# Now print a quick summary
print(f'Shape: {movies_df.shape} \n{movies_df.columns}')
movies_df.head(3)

C:\Users\jcjwg\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape: (25221, 21) 
Index(['title', 'original_title', 'year', 'date_published', 'genre',
       'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'ww_gross_income', 'metascore',
       'user_reviews', 'critic_reviews'],
      dtype='object')


,title,original_title,year,date_published,genre,duration,country,language,director,writer,...,actors,description,avg_vote,votes,budget,usa_gross_income,ww_gross_income,metascore,user_reviews,critic_reviews
title_id,,,,,,,,,,,,,,,,,,,,,
tt0035423,Kate & Leopold,Kate & Leopold,2001,2002-03-01,"Comedy, Fantasy, Romance",118.0,USA,"English, French",James Mangold,"Steven Rogers, James Mangold",...,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",An English Duke from 1876 is inadvertedly drag...,6.4,77852.0,$ 48000000,$ 47121859,$ 76019048,44.0,341.0,115.0
tt0054724,Zombie,Zombie,1971,1971-10-29,Horror,92.0,USA,English,Del Tenney,Del Tenney,...,"William Joyce, Heather Hewitt, Walter Coy, Dan...",A cancer researcher on a remote Caribbean isla...,3.5,1214.0,NaN,NaN,NaN,NaN,56.0,40.0
tt0062690,O Ritual dos Sádicos,O Ritual dos Sádicos,1970,1990-10-01,"Drama, Horror",93.0,Brazil,Portuguese,José Mojica Marins,"Rubens F. Lucchetti, José Mojica Marins",...,"José Mojica Marins, Ângelo Assunção, Ronaldo B...","Psychiatrist experiments LSD on 4 volunteers, ...",6.1,1048.0,NaN,NaN,NaN,NaN,22.0,41.0


#### Further Update Thanks to Jingjing's Work

In [8]:
movies_df.dropna(inplace = True)
#split coloumn 'usa_gross_income', 'worlwode_gross_income' and 'budget' into two columns

movies_df['usa_income_symbol'], movies_df['usa_income_old'] = movies_df['usa_gross_income'].str.split(' ', 1).str
movies_df['usa_income_old'] = movies_df['usa_income_old'].astype(float)

movies_df['worlwide_income_symbol'], movies_df['worlwide_income_old'] = movies_df['ww_gross_income'].str.split(' ', 1).str
movies_df['worlwide_income_old'] = movies_df['worlwide_income_old'].astype(float)

movies_df['budget_symbol'], movies_df['budget_old'] = movies_df['budget'].str.split(' ', 1).str
movies_df['budget_old'] = movies_df['budget_old'].astype(float)

movies_df = movies_df.drop(columns=['usa_gross_income', 'ww_gross_income', 'budget'])

movies_df.head(4)

C:\Users\jcjwg\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  after removing the cwd from sys.path.
C:\Users\jcjwg\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  import sys
C:\Users\jcjwg\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  # Remove the CWD from sys.path while we load stuff.


,title,original_title,year,date_published,genre,duration,country,language,director,writer,...,votes,metascore,user_reviews,critic_reviews,usa_income_symbol,usa_income_old,worlwide_income_symbol,worlwide_income_old,budget_symbol,budget_old
title_id,,,,,,,,,,,,,,,,,,,,,
tt0035423,Kate & Leopold,Kate & Leopold,2001,2002-03-01,"Comedy, Fantasy, Romance",118.0,USA,"English, French",James Mangold,"Steven Rogers, James Mangold",...,77852.0,44.0,341.0,115.0,$,47121859.0,$,76019048.0,$,48000000.0
tt0065134,Gli avvoltoi hanno fame,Two Mules for Sister Sara,1970,1970-12-05,"Adventure, Romance, War",116.0,"USA, Mexico","English, Spanish, French, Latin",Don Siegel,"Budd Boetticher, Albert Maltz",...,23223.0,62.0,79.0,39.0,$,5050000.0,$,5050000.0,$,2500000.0
tt0065377,Airport,Airport,1970,1970-03-25,"Action, Drama, Thriller",137.0,USA,"English, Italian","George Seaton, Henry Hathaway","Arthur Hailey, George Seaton",...,17068.0,42.0,188.0,82.0,$,100489151.0,$,100489151.0,$,10000000.0
tt0065421,Gli Aristogatti,The AristoCats,1970,1971-11-13,"Animation, Adventure, Comedy",78.0,USA,English,Wolfgang Reitherman,"Larry Clemmons, Vance Gerry",...,91085.0,66.0,141.0,113.0,$,35452658.0,$,35459543.0,$,4000000.0


In [4]:
# #currency converter
# from currency_converter import CurrencyConverter
# c = CurrencyConverter()

# movies['usa_income_symbol'] = movies['usa_income_symbol'].str.replace('$','USD')
# movies['usa_income_dollar'] = movies.apply(lambda movies: c.convert(movies['usa_income_old'],movies['usa_income_symbol'],new_currency='USD'),axis=1)

# movies['worlwide_income_symbol'] = movies['worlwide_income_symbol'].str.replace('$','USD')
# movies['worlwide_income_dollar'] = movies.apply(lambda movies: c.convert(movies['worlwide_income_old'],movies['worlwide_income_symbol'],new_currency='USD'),axis=1)

# movies['budget_symbol'] = movies['budget_symbol'].str.replace('$','USD')
# movies['budget_symbol'] = movies['budget_symbol'].str.replace('RUR','RUB')


# def convert_others_to_usd(symbol:str, old_value:float):
#     #currency rates are found in the website: https://www.xe.com/currencyconverter/convert/?Amount=1&From=DEM&To=USD
#     currencies = {
#         'DEM': 0.61261302,
#         'FRF': 0.182659,
#         'ITL': 0.000618802,
#         'BEF': 0.0297018,
#         'ESP': 0.00720113,
#         'ATS': 0.0870742, 
#         'FIM': 0.201517,
#         'CLP': 0.00142245,
#         'NGN': 0.0026280674,
#         'ARS': 0.010775668,
#         'DOP': 0.017537532
#     }
#     if symbol in currencies:
#         converted_value = currencies[symbol] * old_value
#     else:
#         converted_value = c.convert(old_value, symbol, 'USD')
#     return converted_value

        
# movies['budget_dollar'] = movies.apply(lambda movies: convert_others_to_usd(movies['budget_symbol'],movies['budget_old']),axis=1)



# movies.drop(columns = ['usa_gross_income','usa_income_symbol','usa_income_old',
#                        'worlwide_gross_income','worlwide_income_symbol','worlwide_income_old',
#                        'budget','budget_symbol','budget_old'], inplace = True)


### Let the Clustering Begin!